In [17]:
import geopy
from geopy.geocoders import Nominatim
import requests
import folium

## INITIALIZE DATA

In [3]:
drivers = {'Laci': {'address': '1085 Budapest Horánszky utca 6.', 'kapacitas': 4},
           'Feri': {'address': '1153 Budapest Dobó utca 18.', 'capacity': 6}}

In [ ]:
cast_members = {}

In [6]:
geolocator = Nominatim(user_agent="Transport_planner")

In [15]:
laciloc = geolocator.geocode(drivers['Laci']['address'])
print(laciloc.latitude, laciloc.longitude)

47.49236565 19.0677840526337


In [12]:
feriloc = geolocator.geocode(drivers['Feri']['address'])
print(feriloc.latitude, feriloc.longitude)

47.5604898333333 19.1154243333333


## DIRECTIONS API

In [13]:
with open('gitignore/api_key.txt') as fh:
    api_key = fh.read()

In [31]:
url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={str(laciloc.longitude)[:8]},{str(laciloc.latitude)[:8]}&end={str(feriloc.longitude)[:8]},{str(feriloc.latitude)[:8]}"
url

'https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf6248934a0ac43b7847269eec224b84b6f488&start=19.06778,47.49236&end=19.11542,47.56048'

In [65]:
def get_directions(loc_1, loc_2, api_key=api_key):
    # get direction as a geojson
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={str(laciloc.longitude)},{str(laciloc.latitude)}&end={str(feriloc.longitude)},{str(feriloc.latitude)}"
    r = requests.get(url)
    return r.json()

def get_duration_from_geojson(geojson):
    # return time of route plan in seconds
    return geojson['features'][0]['properties']['segments'][0]['duration']

In [64]:
r = requests.get(url)

In [37]:
geojson = r.json()

In [47]:
geojson.keys()

dict_keys(['type', 'features', 'bbox', 'metadata'])

In [61]:
time_in_mins = geojson['features'][0]['properties']['segments'][0]['duration'] / 60

In [62]:
time_in_mins

17.555

In [43]:
m = folium.Map(location=(feriloc.latitude, feriloc.longitude))

folium.GeoJson(
    geojson,
    name='geojson'
).add_to(m)

In [44]:
m